## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,flin flon,CA,1635270917,54.7682,-101.8650,39.04,93,90,8.05,mist
1,1,rikitea,PF,1635270596,-23.1203,-134.9692,75.11,79,100,18.75,light rain
2,2,atuona,PF,1635270604,-9.8000,-139.0333,77.29,68,61,15.90,broken clouds
3,3,saint-ambroise,CA,1635270714,48.5501,-71.3324,55.94,46,100,3.44,overcast clouds
4,4,moron,AR,1635270917,-34.6534,-58.6198,98.02,23,14,1.01,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,rikitea,PF,1635270596,-23.1203,-134.9692,75.11,79,100,18.75,light rain
2,2,atuona,PF,1635270604,-9.8000,-139.0333,77.29,68,61,15.90,broken clouds
7,7,sibolga,ID,1635270918,1.7427,98.7792,75.92,94,100,0.72,overcast clouds
10,10,yulara,AU,1635270919,-25.2406,130.9889,75.04,53,52,3.44,broken clouds
13,13,avarua,CK,1635270795,-21.2078,-159.7750,75.25,64,72,6.91,broken clouds
28,28,butaritari,KI,1635270793,3.0707,172.7902,81.93,77,37,12.82,light rain
36,36,wanning,CN,1635270924,18.8003,110.3967,75.36,91,100,10.13,overcast clouds
38,38,lakatoro,VU,1635270925,-16.0999,167.4164,78.24,85,10,5.88,clear sky
40,40,georgetown,MY,1635270906,5.4112,100.3354,78.89,97,40,7.00,moderate rain
43,43,vanimo,PG,1635270779,-2.6741,141.3028,77.86,89,85,4.59,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             159
City                   159
Country                157
Date                   159
Lat                    159
Lng                    159
Max Temp               159
Humidity               159
Cloudiness             159
Wind Speed             159
Current Description    159
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# clean up the DataFrame by dropna() funtion
clean_preferred_cities_df = preferred_cities_df.dropna()
clean_preferred_cities_df.count()

Unnamed: 0             157
City                   157
Country                157
Date                   157
Lat                    157
Lng                    157
Max Temp               157
Humidity               157
Cloudiness             157
Wind Speed             157
Current Description    157
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,rikitea,PF,75.11,light rain,-23.1203,-134.9692,
2,atuona,PF,77.29,broken clouds,-9.8000,-139.0333,
7,sibolga,ID,75.92,overcast clouds,1.7427,98.7792,
10,yulara,AU,75.04,broken clouds,-25.2406,130.9889,
13,avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
28,butaritari,KI,81.93,light rain,3.0707,172.7902,
36,wanning,CN,75.36,overcast clouds,18.8003,110.3967,
38,lakatoro,VU,78.24,clear sky,-16.0999,167.4164,
40,georgetown,MY,78.89,moderate rain,5.4112,100.3354,
43,vanimo,PG,77.86,moderate rain,-2.6741,141.3028,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")


Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,rikitea,PF,75.11,light rain,-23.1203,-134.9692,People ThankYou
2,atuona,PF,77.29,broken clouds,-9.8000,-139.0333,Villa Enata
7,sibolga,ID,75.92,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
10,yulara,AU,75.04,broken clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
13,avarua,CK,75.25,broken clouds,-21.2078,-159.7750,Paradise Inn


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "VacationPy_database.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))